# Basic scope
In Julia, whenever you are not inside a function, your variables will have global scope. This means they are visible anywhere, so we can do things like this

In [85]:
a = 5
function test(x)
    x + a
end
test(3)

8

In contrast, variables defined inside a function are only visible within that function.

In [86]:
function test1(x)
    b = 2
    x + b
end
test1(2) # b is visible

4

In [87]:
function test2(x)
    x + b
end
test2(2) # It can't find b

UndefVarError: UndefVarError: b not defined

These rules also extend to loops, which, like functions, introduce their own scope blocks. Notably, variables defined inside a scope block do not leave it. This means, as we saw above with `b`, you can't define a variable inside a loop (or a function) and access it outside.

In [88]:
for i = 1:3
    @show i + a # We can see the global a
end

i + a = 6
i + a = 7
i + a = 8


In [89]:
for i = 1:3
    b = 3
    @show i + a
end
b

i + a = 6
i + a = 7
i + a = 8


UndefVarError: UndefVarError: b not defined

You are allowed to modify global variables from inside loops, though doing this is discouraged. You can't modify global variables from inside functions unless you use the global keyword.

In [90]:
function test3(x)
    a = 8
    x + a
end
test3(2)
a # a was not modified

5

In [91]:
function test3(x)
    global a = 8
    x + a
end
test3(2)
a # a was modified

8

In [92]:
s = 0
for i = 1:3
    s += 1
    # The following line does the same thing
    #global s +=1
end
s # s was modified

3

Technical note. The previous example of modifying the global s inside the for loop only works in Jupyter notebooks. This was done in order to make life easier for beginners doing their homework. The previous example would be an error in a regular Julia session (or in Atom), unless you specified the global keyword.

Regardless, it's better style to use the global keyword. Modifying global variables can have unintended side effects later on in your program. 

The QuantEcon notebooks have good style. They use global variables to set things up, but have the main work done inside functions which take arguments. Accessing global variables from inside functions is discouraged in serious work. It's better to be explicit about the variables in scope by using arguments.



# Performance problems of accessing global variables inside functions

As stated earlier, using global variables in a function is considered poor style. It is also very bad for performance.

In [93]:
using Random
ρ = 0.5
function add1(x)
    s = 0.
    for i = 1:100000
        s = ρ*s + x
    end
    s
end

function add2(x, ρ)
    s = 0.
    for i = 1:100000
        s = ρ*s + x
    end
    s
end
@show add1(3)
@show add2(3, ρ); # Same output
@time add1(3)
@time add2(3, ρ) # First version takes much more time

add1(3) = 6.0
add2(3, ρ) = 6.0
  0.003955 seconds (200.00 k allocations: 3.052 MiB)
  0.000290 seconds (5 allocations: 176 bytes)


6.0

This introduces the concept of "type stability" which is a little complicated. The gist is that because $\rho$ in the first function is global, Julia can't guarantee it won't change type somewhere along the way, since global variables can be modified anywhere. Therefore, it has to check the type at run time, instead of when the function is compiled. See https://docs.julialang.org/en/v1/manual/faq/#man-type-stability-1 for a definition of "type-stable" and https://lectures.quantecon.org/jl/more_julia/need_for_speed.html for more information and performance tips.

Be careful about accidentally using global variables in functions. First, you will have performance problems. Second, you may introduce subtle bugs. For example, let's say I had defined a $\alpha$ globally. In my function, I indended to use a parameter $\beta$, but accidentally wrote alpha.

In [96]:
α = 3.
function usebeta(x, β)
    x + α
end
usebeta(1., 50) # The second argument is unused!

4.0

This may seem implausible, but I've experienced similar bugs when working on projects with hundreds of variables.

# Closures

We can define nested functions. These functions can access variables declared in their parent function.

In [94]:
function ctest(x)
    a = 3 + x
    function my_close(y)
        y + a # a is visible
    end
    my_close(5.)
end
ctest(1.)

9.0

The usual reason we do this is because we need to pass a function as an argument, for example, to an optimization routine. This is extremely common in economics. We'll need to solve the same problem for different parameter values.

In [95]:
using Optim
function opttest(x)
    myfun(y) = y^2 - x*y - 2
    optimize(myfun, -10, 10) 
end
opttest(2.) # Minimizes the function y^2 - 2y - 2
opttest(5.) # Minimizes the function y^2 - 5y - 2


Results of Optimization Algorithm
 * Algorithm: Brent's Method
 * Search Interval: [-10.000000, 10.000000]
 * Minimizer: 2.500000e+00
 * Minimum: -8.250000e+00
 * Iterations: 5
 * Convergence: max(|x - x_upper|, |x - x_lower|) <= 2*(1.5e-08*|x|+2.2e-16): true
 * Objective Function Calls: 6